<a href="https://colab.research.google.com/github/estela19/AIFFEL/blob/master/exp02/%5BE-02%5Ddigits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import numpy as np

In [70]:
digits = load_digits()
digits_data = digits.data
digits_label = digits.target

# Data Analysis

In [71]:
print(dir(digits))

['DESCR', 'data', 'images', 'target', 'target_names']


In [72]:
print(digits.DESCR)

.. _digits_dataset:

Optical recognition of handwritten digits dataset
--------------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 5620
    :Number of Attributes: 64
    :Attribute Information: 8x8 image of integer pixels in the range 0..16.
    :Missing Attribute Values: None
    :Creator: E. Alpaydin (alpaydin '@' boun.edu.tr)
    :Date: July; 1998

This is a copy of the test set of the UCI ML hand-written digits datasets
https://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits

The data set contains images of hand-written digits: 10 classes where
each class refers to a digit.

Preprocessing programs made available by NIST were used to extract
normalized bitmaps of handwritten digits from a preprinted form. From a
total of 43 people, 30 contributed to the training set and different 13
to the test set. 32x32 bitmaps are divided into nonoverlapping blocks of
4x4 and the number of on pixels are counted in each blo

iris의 data와 target의 shape를 살펴보면 150개의 데이터셋이 4개의 품종으로 분류 됨을 알 수 있다.

In [73]:
digits_data.shape

(1797, 64)

In [74]:
digits.target.shape

(1797,)

Data Split

In [75]:
X_train, X_test, y_train, y_test = train_test_split(digits_data, 
                                                    digits_label, 
                                                    test_size=0.2, 
                                                    random_state=7)

# 모델 생성 (ensemble: randomforest, svm, logist regression)

In [76]:
decision_tree = DecisionTreeClassifier(random_state=32)
decision_tree.fit(X_train, y_train)
y_pred = decision_tree.predict(X_test)

In [77]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier

from sklearn.metrics import accuracy_score

In [78]:
#개별 모델 생성
svm_ = svm.SVC()
randomforest = RandomForestClassifier(n_estimators = 100, random_state = 0)
logisticreg = LogisticRegression()

In [79]:
#각 모델 학습
svm_.fit(X_train, y_train)
randomforest.fit(X_train, y_train)
logisticreg.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [80]:
svm_pred = svm_.predict(X_train)
rf_pred = randomforest.predict(X_train)
lr_pred = logisticreg.predict(X_train)

In [81]:
new_data = np.array([svm_pred, rf_pred, lr_pred])
new_data.shape

(3, 1437)

In [82]:
new_data = np.transpose(new_data)
new_data.shape

(1437, 3)

In [83]:
#최종 모델
lgbm = LGBMClassifier()

In [84]:
lgbm.fit(new_data, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

# 모델테스트

In [85]:
sp = svm_.predict(X_test)
rp = randomforest.predict(X_test)
lp = logisticreg.predict(X_test)

new_test = np.array([sp, rp, lp])
new_test = np.transpose(new_test)
lgbm_pred = lgbm.predict(new_test)

정확도 측정 (classification_report)

In [86]:
print(classification_report(y_test, lgbm_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        43
           1       0.95      1.00      0.98        42
           2       1.00      1.00      1.00        40
           3       1.00      1.00      1.00        34
           4       1.00      0.97      0.99        37
           5       0.82      1.00      0.90        28
           6       1.00      0.96      0.98        28
           7       0.92      1.00      0.96        33
           8       1.00      0.81      0.90        43
           9       0.97      0.94      0.95        32

    accuracy                           0.97       360
   macro avg       0.97      0.97      0.97       360
weighted avg       0.97      0.97      0.97       360

